### Imports

In [1]:
import os
import sys
import yaml
import argparse
import numpy as np
import torch

from hls4ml.utils.config import config_from_pyg_model
from hls4ml.converters import convert_from_pyg_model
# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml/utils'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from config import config_from_pyg_model

# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from converters import convert_from_pyg_model


from collections import OrderedDict
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# locals
from utils.models.interaction_network_pyg import InteractionNetwork
from model_wrappers import model_wrapper
from utils.data.dataset_pyg import GraphDataset
from utils.data.fix_graph_size import fix_graph_size

layer: Conv1d
func: <function parse_conv1d_layer at 0x7f1eb761a680>
layer: Conv2d
func: <function parse_conv2d_layer at 0x7f1eb761a710>
layer: Linear
func: <function parse_linear_layer at 0x7f1eb761a7a0>
layer: Softmax
func: <function parse_activation_layer at 0x7f1eb761a5f0>
layer: ReLU
func: <function parse_activation_layer at 0x7f1eb761a5f0>
layer: BatchNorm2d
func: <function parse_batchnorm_layer at 0x7f1eb761ab00>
layer: BatchNorm1d
func: <function parse_batchnorm_layer at 0x7f1eb761ab00>
handler args: ('NodeBlock',)
handler args: ('EdgeBlock',)
handler args: ('EdgeAggregate',)
handler args: ('ResidualBlock',)
handler args: ('NodeEncoder',)
handler args: ('EdgeEncoder',)
layer: NodeBlock
func: <function parse_NodeBlock at 0x7f1eb7627ef0>
layer: EdgeBlock
func: <function parse_EdgeBlock at 0x7f1eb762c0e0>
layer: EdgeAggregate
func: <function parse_EdgeAggregate at 0x7f1eb762c290>
layer: ResidualBlock
func: <function parse_ResidualBlock at 0x7f1eb762c440>
layer: NodeEncoder
func: <f

/home/swissman777/anaconda3/envs/hls4ml/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


### PyTorch Model

In [2]:
torch_model = InteractionNetwork(aggr="add", flow="source_to_target", hidden_size=40)
# torch_model_dict = torch.load("trained_models//IN_pyg_small_add_source_to_target_40_state_dict.pt")
# torch_model.load_state_dict(torch_model_dict)

for name, submodule in torch_model.named_modules():
    if name != "":
        print(f"{name}: {submodule}")

O: ObjectModel(
  (layers): Sequential(
    (0): Linear(in_features=5, out_features=40, bias=True)
    (1): ReLU()
    (2): Linear(in_features=40, out_features=40, bias=True)
    (3): ReLU()
    (4): Linear(in_features=40, out_features=5, bias=True)
  )
)
O.layers: Sequential(
  (0): Linear(in_features=5, out_features=40, bias=True)
  (1): ReLU()
  (2): Linear(in_features=40, out_features=40, bias=True)
  (3): ReLU()
  (4): Linear(in_features=40, out_features=5, bias=True)
)
O.layers.0: Linear(in_features=5, out_features=40, bias=True)
O.layers.1: ReLU()
O.layers.2: Linear(in_features=40, out_features=40, bias=True)
O.layers.3: ReLU()
O.layers.4: Linear(in_features=40, out_features=5, bias=True)
R2: RelationalModel(
  (layers): Sequential(
    (0): Linear(in_features=15, out_features=40, bias=True)
    (1): ReLU()
    (2): Linear(in_features=40, out_features=40, bias=True)
    (3): ReLU()
    (4): Linear(in_features=40, out_features=1, bias=True)
  )
)
R2.layers: Sequential(
  (0): Lin

### HLS Model

In [3]:
# forward_dict: defines the order in which graph-blocks are called in the model's 'forward()' method
forward_dict = OrderedDict()
forward_dict["node_encoder"] = "NodeEncoder"
forward_dict["edge_encoder"] = "EdgeEncoder"
# forward_dict["R1"] = "EdgeBlock"
forward_dict["O"] = "NodeBlock"
# forward_dict["res_block"] = "ResidualBlock"
# forward_dict["R2"] = "EdgeBlock"

In [4]:
common_dim = 5
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
#         "node_dim": 3,
        "node_attr": 3,
        "node_dim": common_dim,
#         "edge_dim": 4,
        "edge_attr": 4,
    "edge_dim":common_dim
}

In [5]:
# output_dir = "test_GNN"
# config = config_from_pyg_model(torch_model,
#                                    default_precision="ap_fixed<32,12>",
#                                    default_index_precision='ap_uint<16>', 
#                                    default_reuse_factor=1)
# hls_model = convert_from_pyg_model(torch_model,
#                                        n_edge=graph_dims['n_edge'],
#                                        n_node=graph_dims['n_node'],
#                                        edge_attr=graph_dims['edge_attr'],
#                                        node_attr=graph_dims['node_attr'],
#                                        edge_dim=graph_dims['edge_dim'],
#                                        node_dim=graph_dims['node_dim'],
#                                        forward_dictionary=forward_dict, 
#                                        activate_final='sigmoid',
#                                        output_dir=output_dir,
#                                        hls_config=config)

In [6]:
output_dir = "test_GNN"
config = config_from_pyg_model(torch_model,
                                   default_precision="ap_fixed<64,8>",
                                   default_index_precision='ap_uint<16>', 
                                   default_reuse_factor=8)
print(f"config: {config}")
hls_model = convert_from_pyg_model(torch_model,
                                       n_edge=graph_dims['n_edge'],
                                       n_node=graph_dims['n_node'],
                                       edge_attr=graph_dims['edge_attr'],
                                       node_attr=graph_dims['node_attr'],
                                       edge_dim=graph_dims['edge_dim'],
                                       node_dim=graph_dims['node_dim'],
                                       forward_dictionary=forward_dict, 
                                       activate_final='sigmoid',#'sigmoid',
                                       output_dir=output_dir,
                                       hls_config=config)

config: {'Model': {'Precision': 'ap_fixed<64,8>', 'IndexPrecision': 'ap_uint<16>', 'ReuseFactor': 8, 'Strategy': 'Latency'}}
PygModelReader node_dim: 5
PygModelReader edge_dim: 5
PygModelReader node_attr: 3
PygModelReader edge_attr: 4
node encoder layer_dict['n_in']: 3
node encoder layer_dict['n_out']: 5
edge encoder b4 update_dict["last_edge_update"]: edge_attr
edge encoder after update_dict["last_edge_update"]: layer5_out
type(update_dict): <class 'dict'>
nodeblock layer_dict: {'name': 'O', 'n_node': 28, 'n_edge': 37, 'node_dim': 5, 'edge_dim': 5, 'activate_final': 'false', 'activation': 'linear', 'n_layers': 3, 'out_dim': 5, 'class_name': 'NodeBlock', 'inputs': ['layer4_out', 'layer6_out'], 'outputs': ['layer7_out']}
NodeEncoder name: node_encoder
self.state_dict.keys(): odict_keys(['O.layers.0.weight', 'O.layers.0.bias', 'O.layers.2.weight', 'O.layers.2.bias', 'O.layers.4.weight', 'O.layers.4.bias', 'R2.layers.0.weight', 'R2.layers.0.bias', 'R2.layers.2.weight', 'R2.layers.2.bias',

In [7]:
hls_model.compile()

Writing HLS project
layer: <hls4ml.model.hls_layers.Input object at 0x7f1e9ec4f1d0>
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7f1e9ec4f2d0>])
layer name: node_attr. func: None
layer: <hls4ml.model.hls_layers.Input object at 0x7f1e9ecbc1d0>
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7f1e9ecbc210>])
layer name: edge_attr. func: None
layer: <hls4ml.model.hls_layers.Input object at 0x7f1e9ecbc250>
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7f1e9eca8a10>])
layer name: edge_index. func: None
layer: <hls4ml.model.hls_layers.NodeEncoder object at 0x7f1e9eca8f50>
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7f1e9ec4bf10>])
def_cpp: layer4_t layer4_out[N_LAYER_1_4*N_LAYER_2_4]
layer name: node_encoder. func: ['nnet::encoder<input_t, layer4_t, config4>(node_attr, layer4_out, w4, b4);']
layer: <hls4ml.model.hls_layers.Edg

Done
lib_name: firmware/myproject-DA44e0d4.so


# Evaluation and prediction: hls_model.predict(input)

### Data

In [8]:
class data_wrapper(object):
    def __init__(self, node_attr, edge_attr, edge_index, target):
        self.x = node_attr
        self.edge_attr = edge_attr
        self.edge_index = edge_index.transpose(0,1)

        node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
        node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
        self.hls_data = [node_attr, edge_attr, edge_index]

        self.target = target
        self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs(graph_indir, graph_dims, n_graphs):
    graph_files = np.array(os.listdir(graph_indir))
    graph_files = np.array([os.path.join(graph_indir, graph_file)
                            for graph_file in graph_files])
    n_graphs_total = len(graph_files)
    IDs = np.arange(n_graphs_total)
    print(f"IDS: {IDs}")
    dataset = GraphDataset(graph_files=graph_files[IDs])

    graphs = []
    for data in dataset[:n_graphs]:
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, data.edge_attr, data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
#         if not bad_graph:
#             graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
    print(f"graphs length: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
# graph_dims = {
#         "n_node": 28,
#         "n_edge": 37,
#         "node_dim": 3,
#         "edge_dim": 4
#     }
graphs = load_graphs(graph_indir, graph_dims, n_graphs=10)

IDS: [0 1 2 3 4 5 6 7 8 9]
graphs length: 10
writing test bench data for 1st graph


In [9]:
data = graphs[0]
print(f"graphs len: {len(graphs)}")
print(type(data.hls_data))
print(f"data.x shape:{data.x.shape}")
torch_pred = torch_model(data)
torch_pred = torch_pred.detach().cpu().numpy().flatten()
print(f"torch pred shape: {torch_pred.shape}")
hls_pred = hls_model.predict(data.hls_data)
print(f"hls_pred.shape: {hls_pred.shape}")
MSE = mean_squared_error(torch_pred, hls_pred)
print(f"MSE: {MSE}")

print((np.abs(torch_pred- hls_pred)))
print(torch_pred)
print(hls_pred)

graphs len: 10
<class 'list'>
data.x shape:torch.Size([28, 3])
torch pred shape: (140,)
hls_pred.shape: (140,)
MSE: 5.532775503525045e-06
[1.4348626e-03 2.7398467e-03 2.2478700e-03 2.9376745e-03 5.0061941e-04
 2.7555227e-04 3.2572150e-03 2.9924810e-03 3.3681989e-03 1.8061697e-03
 3.3860207e-03 2.6648641e-03 3.2416284e-03 1.3026595e-03 1.8168092e-03
 4.1339993e-03 2.9626787e-03 1.7241836e-03 2.8865337e-03 1.7311871e-03
 1.0411739e-03 4.0471554e-04 5.3942204e-04 3.2474399e-03 8.7320805e-04
 1.7563105e-03 2.5377870e-03 8.8855624e-04 3.1298399e-04 1.5601516e-04
 4.1246414e-03 1.4972985e-03 2.5479198e-03 3.7403107e-03 1.5625358e-04
 4.9614906e-04 3.3468008e-05 1.9069612e-03 2.0247102e-03 1.6623437e-03
 3.1077862e-03 4.7802925e-04 3.1187832e-03 1.9257069e-03 1.4294386e-03
 3.2761693e-03 2.9364824e-03 9.6195936e-04 3.8756728e-03 2.3882687e-03
 2.5613308e-03 3.2360256e-03 3.1945109e-03 3.4933090e-03 3.4992099e-03
 3.4584999e-03 2.3280680e-03 2.2519231e-03 2.8617680e-03 2.8455257e-04
 3.0028820

In [10]:
def get_meanNstd(data, torch_pred, hls_pred):
#     print(data.x)
    node_mean = torch.mean(data.x)
    print(f"node_mean: {node_mean}")
    node_std = torch.std(data.x)
    print(f"node_std: {node_std}")
    node_abs_max = torch.max(torch.abs(data.x))
    print(f"node_abs_max: {node_abs_max}")
    edge_mean = torch.mean(data.edge_attr)
    print(f"edge_mean: {edge_mean}")
    edge_std = torch.std(data.edge_attr)
    print(f"edge_std: {edge_std}")
    edge_abs_max = torch.max(torch.abs(data.edge_attr))
    print(f"edge_abs_max: {edge_abs_max}")
    torch_pred_mean = np.mean(torch_pred)
    print(f"torch_pred_mean: {torch_pred_mean}")
    torch_pred_std = np.std(torch_pred)
    print(f"torch_pred_std: {torch_pred_std}")
    torch_pred_abs_max = np.max(np.abs(torch_pred))
    print(f"torch_pred_abs_max: {torch_pred_abs_max}")
    hls_pred_mean = np.mean(hls_pred)
    print(f"hls_pred_mean: {hls_pred_mean}")
    hls_pred_std = np.std(hls_pred)
    print(f"hls_pred_std: {hls_pred_std}")
    hls_pred_abs_max = np.max(np.abs(hls_pred))
    print(f"hls_pred_abs_max: {hls_pred_abs_max}")
    diff_mean = np.mean(torch_pred- hls_pred)
    print(f"diff_mean: {diff_mean}")
    diff_std = np.std(torch_pred- hls_pred)
    print(f"diff_std: {diff_std}")
    diff_abs_max = np.max(np.abs(torch_pred- hls_pred))
    print(f"diff_abs_max: {diff_abs_max}")
    return None

In [11]:
MSE_l = []
for data in graphs:
#     print(f"graphs len: {len(graphs)}")
#     print(type(data.hls_data))
#     print(f"data.x shape:{data.x.shape}")
    torch_pred = torch_model(data)
    torch_pred = torch_pred.detach().cpu().numpy().flatten()
#     print(f"torch pred shape: {torch_pred.shape}")
    hls_pred = hls_model.predict(data.hls_data)
#     print(f"hls_pred.shape: {hls_pred.shape}")
    MSE = mean_squared_error(torch_pred, hls_pred)
#     get_meanNstd(data, torch_pred, hls_pred)
    print(f"MSE: {MSE}")
    MSE_l.append(MSE)
#     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
#     print(f"MAPE: {MAPE}")

MSE_l = np.array(MSE_l)
print(f"MSE_l: {MSE_l}")
print(f"overall MSE: {np.mean(MSE_l)}")
print(MSE_l.shape)

MSE: 5.532775503525045e-06
MSE: 6.2269864429254085e-06
MSE: 8.010804776859004e-06
MSE: 7.520029612351209e-06
MSE: 5.9222938944003545e-06
MSE: 5.2082295951549895e-06
MSE: 6.491612566605909e-06
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_laye

In [12]:
for data_instance in data.hls_data:
    print(data_instance.shape)
    
print(data.edge_index.shape)

(28, 3)
(37, 4)
(37, 2)
torch.Size([2, 37])


In [13]:
# class data_wrapper_tau3mu:
#     def __init__(self, node_attr, edge_attr, edge_index, target):
#         self.x = node_attr
#         self.edge_attr = edge_attr
#         self.edge_index = edge_index.transpose(0,1)

#         node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
#         node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
#         self.hls_data = [node_attr, edge_attr, edge_index]

#         self.target = target
#         self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs_tau3mu(data_loader, graph_dims:dict, n_graphs =1000):
    """
    params:
    dataloader: pyg dataloader with custom Tau3MuDataset as its dataset
    graph_dims: 
        graph_dims.keys = ["n_node": max number of nodes allowed in a graph/batch,
            "n_edge": max number of edges allowed in a graph/batch,
            "node_dim": feature dim of node,
            "edge_dim": feature dim of edge
        ]
    
    """

    graphs = []
    
    i = 0
    for data in data_loader:
        if i >= n_graphs:
            break
        data.edge_index = data.edge_index.t()#transpose the edge_index
        n_edges = data.edge_attr.shape[0]
#         print((data.y.shape))
#         print(n_edges)
        data.y = data.y.expand(1, n_edges).flatten()
#         print(f"data.y.shape: {data.y.shape}")
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, 
                                                                             data.edge_attr, 
                                                                             data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
        target = torch.flatten(target)# flatten target to 1d array
        if not bad_graph:
            graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        i +=1
        
    print(f"n_graphs: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
# graph_indir = "/home/swissman777/projects/Tau3MuGNNs/data"
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
        "node_dim": 3,
        "edge_dim": 4
    }


In [14]:
import pickle as pkl
from torch_geometric.loader.dataloader import DataLoader
from utils.dataset import Tau3MuDataset

# with open('./trackml_data/data_loaders.pickle', 'rb') as file:
with open('./trackml_data/data_loaders_batch_size_1.pickle', 'rb') as file:
    data_loaders= pkl.load(file) #, x_dim, edge_attr_dim 

print(data_loaders)
# for stage in data_loaders.keys():
#     for data in data_loaders[stage]:
#         print(data.x.shape)
#         print(data.edge_index.shape)
#         print(data.edge_attr.shape)

# print(graphs)

{'train': <torch_geometric.loader.dataloader.DataLoader object at 0x7f1e9ebc9c90>, 'valid': <torch_geometric.loader.dataloader.DataLoader object at 0x7f1e9ec31b90>, 'test': <torch_geometric.loader.dataloader.DataLoader object at 0x7f1e9ec14510>}


In [15]:
# from sklearn.preprocessing import StandardScaler
MSE_l = []
stages = ["train", "valid", "test"]
# node_scaler = StandardScaler()
# edge_scaler = StandardScaler()
for stage in stages:
    graphs = load_graphs_tau3mu(data_loaders[stage], graph_dims)
#     print(type(graphs))

#     for data in graphs: # fit data to scaler b4 feedforwarding
# #         print(f"data type: {type(data)}")
#         node_scaler.fit(data.x) 
#         edge_scaler.fit(data.edge_attr) 
    
    for data in graphs:
       
#         data.x = torch.from_numpy(node_scaler.transform(data.x)).float()
#         data.edge_attr = torch.from_numpy(edge_scaler.transform(data.edge_attr)).float()
#         data.hls_data[0] = node_scaler.transform(data.hls_data[0])
#         data.hls_data[1] = edge_scaler.transform(data.hls_data[1])
        
        torch_pred = torch_model(data)
        torch_pred = torch_pred.detach().cpu().numpy()
#         print(f"torch_pred shape: {torch_pred.shape}")
        hls_pred = hls_model.predict(data.hls_data)
#         print(f"hls_pred shape: {hls_pred.shape}")
        MSE = mean_squared_error(torch_pred, hls_pred)
        print(f"MSE: {MSE}")
    #     get_meanNstd(data,torch_pred, hls_pred)
    #     print(f"MSE: {MSE}")
        MSE_l.append(MSE)
    #     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
    #     print(f"MAPE: {MAPE}")
MSE_l = np.array(MSE_l)
print(f"MSE_l :{MSE_l}")
print(f"MSE means: {np.mean(MSE_l)}")

n_graphs: 1000
writing test bench data for 1st graph
MSE: 0.0856553390622139
MSE: 0.10156439989805222
MSE: 0.037386875599622726
MSE: 0.0713653564453125
MSE: 0.11654099076986313
MSE: 0.07838466018438339
 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layer

MSE: 0.054902054369449615
 == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 

MSE: 0.04942629858851433
MSE: 0.15285199880599976
MSE: 0.10511477291584015
MSE: 0.052430860698223114
ers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_la

MSE: 0.03568547964096069
MSE: 0.09728839993476868
MSE: 0.10982081294059753
MSE: 0.09132907539606094
MSE: 0.036277543753385544
MSE: 0.034961502999067307
MSE: 0.05785231664776802
layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
Encoder starting with n_in: 3 
E

MSE: 0.05539621412754059
layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layer

...

MSE: 0.1526932567358017
MSE: 0.07935138791799545
MSE: 0.12700892984867096
MSE: 0.06802377849817276
MSE: 0.1847381591796875
MSE: 0.09188485145568848
 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_lay

MSE: 0.1832560896873474
MSE: 0.09337171167135239
MSE: 0.0660061165690422
 == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layer

MSE: 0.04355515539646149
MSE: 0.16984568536281586
MSE: 0.017557863146066666
MSE: 0.041699666529893875
MSE: 0.08237093687057495
rting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 

MSE: 0.04708300158381462
MSE: 0.07352995127439499
MSE: 0.10309913754463196
MSE: 0.12523868680000305
MSE: 0.04036286473274231
MSE: 0.05473681911826134
MSE: 0.062090855091810226
ers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3

MSE: 0.10385534167289734
MSE: 0.1319791078567505
MSE: 0.05385977774858475
MSE: 0.05525558441877365
MSE: 0.09129942208528519
MSE: 0.06057846173644066
MSE: 0.06374561041593552

n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Nodeblock start 
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_layers == 3
n_la

In [16]:
MSE_l.shape

(3000,)

In [17]:
torch_model.state_dict()

OrderedDict([('O.layers.0.weight',
              tensor([[-0.4188,  0.3944,  0.0674, -0.4437, -0.3974],
                      [ 0.2279, -0.3338,  0.0219, -0.2457,  0.0817],
                      [ 0.4357, -0.2686, -0.0411, -0.1001, -0.4401],
                      [-0.0873,  0.1861,  0.2652,  0.3387,  0.0980],
                      [ 0.1576, -0.4190, -0.1868,  0.2331,  0.1130],
                      [ 0.2962, -0.3531, -0.2542,  0.2103, -0.2802],
                      [ 0.1935, -0.1258,  0.4343, -0.3988,  0.2858],
                      [ 0.3353,  0.2503, -0.3920, -0.0052,  0.1640],
                      [-0.0542, -0.0300, -0.2229, -0.1301,  0.2129],
                      [ 0.1966, -0.1010, -0.3282, -0.1450, -0.0920],
                      [-0.1952,  0.3250,  0.0143, -0.3512, -0.1762],
                      [ 0.0806, -0.1457,  0.4167,  0.0968,  0.2774],
                      [ 0.3493,  0.3450, -0.2313, -0.1261, -0.2980],
                      [ 0.1752, -0.0946, -0.1415, -0.2863, -0.2489],